In [ ]:
!pip install fastai==2.6

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import fastai; fastai.__version__

'2.6.0'

In [ ]:
import pandas as pd
from fastai.tabular.all import *

In [ ]:
# the datasets
house_price_df = pd.read_csv("/content/drive/MyDrive/QUTREProject/Brisbane_suburbs_02_sales_2018_property_GB_Prop_dist16.csv", index_col=0)
house_price_df

In [ ]:
 df_01 = house_price_df.loc[house_price_df['school_dist'] > 10000]
 df_01

In [ ]:
house_price_df = house_price_df.drop(df_01.index)
house_price_df

In [ ]:
house_price_df.reset_index(drop=True, inplace=True)
house_price_df

In [ ]:
# test_df = house_price_df.loc[(house_price_df['suburb_id_x']=='QLD1346') | (house_price_df['suburb_id_x']=='QLD627') | (house_price_df['suburb_id_x']=='QLD24') | \
#                          (house_price_df['suburb_id_x']=='QLD3231') | (house_price_df['suburb_id_x']=='QLD32') | (house_price_df['suburb_id_x']=='QLD510') | \
#                          (house_price_df['suburb_id_x']=='QLD545') | (house_price_df['suburb_id_x']=='QLD718') | (house_price_df['suburb_id_x']=='QLD389') ]
# test_df

In [ ]:
train_df=house_price_df.sample(frac=0.8,random_state=200) #random state is a seed value
test_df=house_price_df.drop(train_df.index)
# train_df=house_price_df.drop(test_df.index)
train_df.reset_index()
test_df.reset_index()

In [ ]:
train_df.reset_index(drop=True, inplace=True)
train_df

In [ ]:
test_df.reset_index(drop=True, inplace=True)
test_df

In [ ]:
# Check the length of the dataset
print(train_df.shape)
print(test_df.shape)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
print(train_df.columns)

In [ ]:
train_df.describe()

In [ ]:
train_df.isnull().sum()

In [ ]:
# Let's check for the test set as well
test_df.isnull().sum()

In [ ]:
# Preprocessing
procs = [Categorify, FillMissing, Normalize]
#procs = [FillMissing, Normalize]

In [ ]:
# Split our variables into target, categorical and continuous variables
dep_var = 'price_value'

cat_names = ['suburb_id_x']
# cat_names = ['street_id_x']
# cat_names = ['street_id_x', 'suburb_id_x']

# cat_names = []
cont_names = ['propertyFeatures_bedrooms_x', 'propertyFeatures_bathrooms_x', 'propertyFeatures_carparks_x', 'landDetails_propertyArea_x', 'school_dist'\
              , 'hospital_dist', 'uni_dist', 'park_dist', 'mall_dist', 'CBD_dist', 'First_year_transaction', 'highway_dist', 'bus_stop_dist', \
       'ferry_terminal_dist', 'railway_station_dist', 'number_transaction', \
       'crime_rate_2017', 'state_school_percentage', \
       'class_size_achievement_pct_2017', 'Density', 'Score2017']

y_block = RegressionBlock()

print(cat_names)
print(cont_names)

In [ ]:
splits = RandomSplitter()(range_of(train_df))
to = TabularPandas(train_df, procs=procs, cat_names=cat_names, cont_names=cont_names,
                   y_names=dep_var, y_block=y_block, splits=splits)

In [ ]:
dls = to.dataloaders()

In [ ]:
# trn_dl = TabDataLoader(to.train, bs=64, shuffle=True, drop_last=True)
# val_dl = TabDataLoader(to.valid, bs=128)

In [ ]:
# dls = DataLoaders(trn_dl, val_dl)

In [ ]:
dls.show_batch()

In [ ]:
# emb_szs = get_emb_sz(to); emb_szs
# cont_len = len(to.cont_names); cont_len

# net = TabularModel(emb_szs, cont_len, 1, [200,100])

In [ ]:
# net

In [ ]:
learn = tabular_learner(dls, [200,100], metrics=rmse, loss_func=MSELossFlat())

In [ ]:
learn.lr_find()

In [ ]:
learn.summary()

In [ ]:
learn.fit(10, 1e-2)
# learn.fit_one_cycle(5, 3e-3, wd=0.2)

In [ ]:
learn.model_dir = '/content/drive/MyDrive/Fastai/'

In [ ]:
learn.save('myModel')
# del learn
learn.load('myModel')

In [ ]:
test_dl = learn.dls.test_dl(test_df)

In [ ]:
test_df

In [ ]:
learn.validate(dl=test_dl)

In [ ]:
# raw_test_preds = learn.get_preds(test_dl)
inp, preds,_,dec_preds = learn.get_preds(dl=test_dl, with_input=True, with_decoded=True)
# inp, preds= learn.get_preds(dl=test_dl)
# preds = learn.get_preds(dl=test_dl)

In [ ]:
learn.model.embeds

In [ ]:
print(preds)

In [ ]:
# variable = 'suburb_id_x'
# ix = cat_names.index(variable)

# var_vals = list(df[variable].astype('category').cat.categories.values)
# nval = len(var_vals)
# print(f'Number of values: {nval}')
# print(var_vals)

In [ ]:
# import altair as alt
# from sklearn.decomposition import PCA
# from sklearn.manifold import TSNE

# emb_mx = to_np(next(learn.model.embeds[ix].parameters()))
# # X_emb = TSNE(n_components=2, perplexity=3).fit_transform(emb_mx)
# X_emb = PCA(n_components=2).fit_transform(emb_mx)
# # annotation = np.append('Other', np.array(var_vals))
# annotation = np.array(var_vals)

In [ ]:
# print(annotation)

In [ ]:
# emb_df = pd.DataFrame(X_emb, columns=['Dim1', 'Dim2'])
# emb_df[variable]=annotation

In [ ]:
# points = alt.Chart(emb_df).mark_circle(size=60).encode(
#     x='Dim1',
#     y='Dim2',
#     tooltip=[variable]
# )

# text = points.mark_text(
#     align='left',
#     baseline='middle',
#     dx=7
# ).encode(
#     text=variable
# )

# points + text

In [ ]:
# print(preds)
df2 = pd.DataFrame(data=preds, columns=['Predicted'])
df2

In [ ]:
# import numpy as np

# #Converting the tensor output to a list of predicted values
# # test_predictions = [np.exp(p[0].data.item()) for p in test_predictions]
# # test_predictions = [(p[0].data.item()) for p in test_predictions]


# # Create "submission.csv" file
# submission = pd.DataFrame({'Id': test_df['id_combined'], 'price_value': test_df['price_value'], 'Predicted': preds})
# submission.to_csv('submission.csv', index=False)
# submission.head()

In [ ]:
import pandas
import matplotlib.pyplot as plt
import statsmodels.api
import statsmodels.formula.api as sm
import scipy.stats as stats

In [ ]:
fig = plt.figure(figsize=[10, 10])
ax = fig.add_subplot(2, 2, 1)
ax.scatter(submission['price_value'], submission['Predicted'])
ax.set_xlabel('value')
ax.set_ylabel('predicted')
ax.set_title(str(submission['Predicted'].corr(submission['price_value'])))

In [ ]:
import numpy as np
Y_true = submission['price_value']
pred = submission['Predicted']


#RMSLE
error = np.square(np.log(pred + 1) - np.log(Y_true +1)).mean() ** 0.5
print(error)

score = 1 - error
print("SCORE For test : ",score)


In [ ]:
dfCopy = test_df.copy()
dfCopy

In [ ]:
dfCopy['Predicted'] = test_predictions
dfCopy

In [ ]:
# dfCopy.to_csv('/content/drive/MyDrive/QUTREProject/2018_Hedonic_Pricing_Test_results_new2.csv')
# dfCopy.to_pickle('/content/drive/MyDrive/QUTREProject/2018_Hedonic_Pricing_Test_results_new2.pkl')